In [2]:
import sys
import json, os, re, shutil, sys, time
import collections, itertools
import pickle

from random import randint

from gensim.models import word2vec, KeyedVectors
from sklearn.cross_validation import train_test_split
from sklearn.metrics import precision_recall_fscore_support

import numpy as np
import tensorflow as tf
assert(tf.__version__.startswith("0.12"))

interim_path = '../../interim/'

import rnnlm
import util

reload(rnnlm)
reload(util)

print "Done!"

Done!


In [5]:
samp = True
load_word2vec=False
samp = '_samp' if samp else ''

In [4]:
# load preprocessed data array
#   podcast+array[0]: podcast name
#   podcast_array[1]: list of words in concatated descriptions, cleaned 
#   podcast_array[2]: subgenre (label)
with open( interim_path + 'preprocessed_concatenated_all' + samp + '.p') as p:
	podcast_array = pickle.load(p)

print podcast_array.shape

(1241, 3)


In [4]:
if load_word2vec:
    start = time.time()
    w2vmodel = KeyedVectors.load_word2vec_format( interim_path + 'GoogleNews-vectors-negative300.bin', binary=True)
    #w2vmodel.save('../interim/gensim_model')
    #w2vmodel = KeyedVectors.load('../interim/gensim_model')
    print w2vmodel.most_similar('dog')
    print w2vmodel['dog'].shape 
    print util.elapsed_time(start, time.time())

[(u'dogs', 0.8680489659309387), (u'puppy', 0.8106428384780884), (u'pit_bull', 0.780396044254303), (u'pooch', 0.7627377510070801), (u'cat', 0.7609456777572632), (u'golden_retriever', 0.7500902414321899), (u'German_shepherd', 0.7465174198150635), (u'Rottweiler', 0.7437614798545837), (u'beagle', 0.7418621778488159), (u'pup', 0.740691065788269)]
(300,)
Time elapsed: 01:05.034479


In [6]:
# collect the 300-element word vectors per description
# into a single list and store in a file
# 20 seconds with 16 CPUs
# doesn't complete with 4 CPUs
if load_word2vec:
    start = time.time()
    documents = [] # list of documents
    for podcast in podcast_array:
        word_vecs = [] # list of embedments for the words in the document
        document = podcast[1]
      
        if document is not None:
            for j in range(0, min(20000, len(document))):
                word = document[j]
                if word in w2vmodel:
                    word_vecs.append(w2vmodel[word])

        documents.append(np.array(word_vecs))
        idx+=1
    print util.elapsed_time(start, time.time())


In [6]:
if load_word2vec:
    # save the document vectors    
    filepath=interim_path + 'preprocessed_concatenated_document_vector_arrays' + samp + '.p'
    with open(filepath, 'wb') as f:
        pickle.dump(documents,f)
    new_doc_vector_list = documents
    print "using newly created list", len(new_doc_vector_list)    

using newly created list 1241


In [7]:
if not load_word2vec:
    filepath = interim_path + 'preprocessed_concatenated_document_vector_arrays' + samp + '.p'
    with open(filepath, 'rb') as f:
        new_doc_vector_list = pickle.load(f)
    print "list loaded from file", len(new_doc_vector_list)

list loaded from file 1241


In [15]:
# break out train and test 
# x_train and x_dev are 1d arrays containing 2d arrays representing the word embedments for the document
vl = np.array(new_doc_vector_list)
x_train, x_dev, y_train, y_dev = train_test_split(vl, podcast_array[:,[1]], test_size=0.25, random_state=42)
print x_train.shape, x_dev.shape, y_train.shape, y_dev.shape

(930,) (311,) (930, 1) (311, 1)


In [25]:
def run_epoch(lm, session, batch_iterator,
              train=False, verbose=False,
              tick_s=10, learning_rate=0.1):
    start_time = time.time()
    tick_time = start_time  # for showing status
    total_cost = 0.0  # total cost, summed over all words
    total_batches = 0
    total_words = 0

    if train:
        train_op = lm.train_step_
        use_dropout = True
        loss = lm.train_loss_
    else:
        train_op = tf.no_op()
        use_dropout = False  # no dropout at test time
        loss = lm.loss_  # true loss, if train_loss is an approximation

    for i, (w, y) in enumerate(batch_iterator):
        cost = 0.0
        # At first batch in epoch, get a clean intitial state.
        if i == 0:
            h = session.run(lm.initial_h_, {lm.input_w_: w})

        feed_dict = { lm.input_w_: w, 
                      lm.target_y_: y, 
                      lm.initial_h_ : h, 
                      lm.learning_rate_ : learning_rate, 
                      lm.use_dropout_ : use_dropout 
                    }
        
        print 'feed_dict dump -- w.shape:', w.shape, 'y.shape:', y.shape, 'h.shape:', h[0][0].shape
        
        h, cost, _ = session.run( [lm.final_h_, loss, train_op], feed_dict=feed_dict )
        print 'foo'
        total_cost += cost
        total_batches = i + 1
        total_words += w.size  # w.size = batch_size * max_time

        # Print average loss-so-far for epoch
        # If using train_loss_, this may be an underestimate.
        if verbose and (time.time() - tick_time >= tick_s):
            avg_cost = total_cost / total_batches
            avg_wps = total_words / (time.time() - start_time)
            print "[batch %d]: seen %d vectors at %d vps, loss = %.3f" % (
                i, total_words, avg_wps, avg_cost)
            tick_time = time.time()  # reset time ticker

    return total_cost / total_batches

In [15]:
def score_dataset(lm, session, ids, name="Data"):
    # For scoring, we can use larger batches to speed things up.
    bi = utils.batch_generator(ids, batch_size=100, max_time=100)
    cost = run_epoch(lm, session, bi, 
                     learning_rate=1.0, train=False, 
                     verbose=True, tick_s=3600)
    print "%s: avg. loss: %.03f  (perplexity: %.02f)" % (name, cost, np.exp(cost))

In [27]:
reload(rnnlm)
reload(util)



<module 'util' from 'util.pyc'>

In [26]:
reload(rnnlm)
reload(util)

x_data = x_train[0]

# training parameters
max_time = 20
batch_size = 50
learning_rate = 0.5
num_epochs = 5

# Model parameters
V=len(x_data)
H=len(x_data[0]) *2
model_params = dict(V=V, H=H, softmax_ns=200, num_layers=1)

print model_params

TF_SAVEDIR = "tf_saved"
checkpoint_filename = os.path.join(TF_SAVEDIR, "rnnlm")
trained_filename = os.path.join(TF_SAVEDIR, "rnnlm_trained")

# Will print status every this many seconds
print_interval = 15

# Clear old log directory
shutil.rmtree("tf_summaries", ignore_errors=True)

lm = rnnlm.RNNLM(**model_params)
lm.BuildCoreGraph()
lm.BuildTrainGraph()

# Explicitly add global initializer and variable saver to LM graph
with lm.graph.as_default():
    initializer = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
# Clear old log directory
shutil.rmtree(TF_SAVEDIR, ignore_errors=True)
if not os.path.isdir(TF_SAVEDIR):
    os.makedirs(TF_SAVEDIR)

with tf.Session(graph=lm.graph) as session:
    # Seed RNG for repeatability
    tf.set_random_seed(42)

    session.run(initializer)

    for epoch in xrange(1,num_epochs+1):
        t0_epoch = time.time()
        bi = util.batch_generator(x_data, batch_size, max_time)
        print "[epoch %d] Starting epoch %d" % (epoch, epoch)
        
        run_epoch(lm, session, bi,
              train=True, verbose=True,
              tick_s=print_interval, learning_rate=learning_rate)
        
        print "[epoch %d] Completed in %s" % (epoch, util.pretty_timedelta(since=t0_epoch))
    
        # Save a checkpoint
        saver.save(session, checkpoint_filename, global_step=epoch)
    
        ##
        # score_dataset will run a forward pass over the entire dataset
        # and report perplexity scores. This can be slow (around 1/2 to 
        # 1/4 as long as a full epoch), so you may want to comment it out
        # to speed up training on a slow machine. Be sure to run it at the 
        # end to evaluate your score.
        print ("[epoch %d]" % epoch),
        score_dataset(lm, session, x_train, name="Train set")
        print ("[epoch %d]" % epoch),
        score_dataset(lm, session, x_dev, name="Test set")
        print ""
    
    # Save final model
    saver.save(session, trained_filename)

{'H': 600, 'num_layers': 1, 'softmax_ns': 200, 'V': 2543}
[epoch 1] Starting epoch 1
feed_dict dump -- w.shape: (50, 20) y.shape: (50, 20) h.shape: (50, 600)


InvalidArgumentError: indices[158] = -1 is not in [0, 2543)
	 [[Node: per_example_sampled_softmax_loss/embedding_lookup_1 = Gather[Tindices=DT_INT64, Tparams=DT_FLOAT, _class=["loc:@recurrent_layer/Variable"], validate_indices=true, _device="/job:localhost/replica:0/task:0/cpu:0"](recurrent_layer/Variable/read, per_example_sampled_softmax_loss/concat)]]

Caused by op u'per_example_sampled_softmax_loss/embedding_lookup_1', defined at:
  File "/home/michaelrubin/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/michaelrubin/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/michaelrubin/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/michaelrubin/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 653, in launch_instance
    app.start()
  File "/home/michaelrubin/ana